In [1]:
# get input file name (.xlsx)
def get_input_excel_name():
    return input("Enter the input file name (with extension .xlsx): ").strip()

In [2]:
# get output file name (.mm)
def get_output_xml_name():
    return input("Enter the output file name (with extension .mm): ").strip()


In [3]:
import tkinter as tk
from tkinter import messagebox

# message box for folder creation will pop up to give the user instructions to follow
def warningbox_creat_folder():
    root=tk.Tk()
    root.withdraw()
    messagebox.showinfo("Folder being created", "Now, move your input files xlsx in the new folder before shuting down the warning box")
    root.destroy()

In [4]:
import tkinter as tk
from tkinter import messagebox

# message box for renaming the folder will pop up to give the user instructions to follow
def warningbox_rename_folder():
    root=tk.Tk()
    root.withdraw()
    messagebox.showinfo("Folder being renamed", "Now, make sure you have your input files xlsx in this folder before shuting down the warning box")
    root.destroy()

In [5]:
# function to create a folder
from pathlib import Path

def create_folder(base_path, folder_name):
    folder_path= Path(base_path)/folder_name
    if not folder_path.exists():
        folder_path.mkdir(parents=True)
        print(f"Folder created: {folder_path}")
        warningbox_creat_folder()
    else:
        print(f"Folder already exists: {folder_path}")
    return folder_path

In [6]:
# function to rename a folder

def rename_folder(base_path, old_name, new_name):
    old_path = Path(base_path) / old_name
    new_path = Path(base_path) / new_name
    
    if old_path.exists():
        old_path.rename(new_path)
        print(f"Folder renamed from {old_name} to {new_name}")
        warningbox_rename_folder()
        return new_path
    else:
        print(f"Folder '{old_name}' does not exist.")
        return None


In [7]:
# function to manage the input path
def main():
    home_dir = Path.home()  # ton home directory C/User/ton NNI
    subdir_path = home_dir / "OneDrive - EDF" / "Data_input_for_studies" / "xmind_tool"
    subdir_path.mkdir(parents=True, exist_ok=True)
    print(f"Using subdirectory path: {subdir_path}")

    # spécifier ou changer le nom du dossier
    action = input("Do you want to create or rename a folder? (create/rename/none): ").strip().lower()
    if action == "create":
        folder_name = input("Enter the name of the new folder: ").strip()
        folder_path = create_folder(subdir_path, folder_name)
    elif action == "rename":
        old_name = input("Enter the current folder name: ").strip()
        new_name = input("Enter the new folder name: ").strip()
        folder_path = rename_folder(subdir_path, old_name, new_name)
    else:
        folder_name = "DefaultFolder"
        folder_path = subdir_path / folder_name
        print(f"Using default folder path: {folder_path}")
        folder_path.mkdir(parents=True, exist_ok=True)

    # indiquer un nom à ton input fichier (xmind)
    input_file_name = get_input_excel_name()
    input_file_path = folder_path / input_file_name

    print(f"Constructed file path: {input_file_path}")

    if not input_file_path.exists():
        with open(input_file_path, 'w') as file:
            file.write("")  # Create an empty file
        print(f"File created: {input_file_path}")
    else:
        print(f"File already exists: {input_file_path}")

    # indiquer un nom à ton output fichier (xlsx)
    output_file_name=get_output_xml_name()
    output_file_path=folder_path / output_file_name

    print(f"Constructed file path: {output_file_path}")

    # if not output_file_path.exists():
    #     with open(output_file_path, 'w') as file:
    #         file.write("")
    #     print(f"Output file created: {output_file_path}")
    # else:
    #     print(f"Output file already exists: {output_file_path}")
    
    # parsed_data = xmind_to_dict(input_file_path)
    return input_file_path, output_file_path


if __name__ == "__main__":
    input_file_path, output_file_path = main()
    print(input_file_path, output_file_path)


Using subdirectory path: C:\Users\jl0331el\OneDrive - EDF\Data_input_for_studies\xmind_tool
Folder created: C:\Users\jl0331el\OneDrive - EDF\Data_input_for_studies\xmind_tool\wahaha
Constructed file path: C:\Users\jl0331el\OneDrive - EDF\Data_input_for_studies\xmind_tool\wahaha\exported_data.xlsx
File already exists: C:\Users\jl0331el\OneDrive - EDF\Data_input_for_studies\xmind_tool\wahaha\exported_data.xlsx
Constructed file path: C:\Users\jl0331el\OneDrive - EDF\Data_input_for_studies\xmind_tool\wahaha\yeeha.mm
C:\Users\jl0331el\OneDrive - EDF\Data_input_for_studies\xmind_tool\wahaha\exported_data.xlsx C:\Users\jl0331el\OneDrive - EDF\Data_input_for_studies\xmind_tool\wahaha\yeeha.mm


In [8]:
# transformer en xml
import pandas as pd
import xml.etree.ElementTree as ET


def create_node(parent, text, link=None, color=None, style=None):
    """
    créer un noeud pour freeMind, ajouter url
    """
    attributes = {"TEXT": text}
    if link:
        attributes["LINK"] = link  # considérer URL comme un attribut
    if color:
        attributes["COLOR"] = color  # paramétrer la couleur des noeuds
    if style:
        attributes["STYLE"] = style  # paramétrer le style des liens
    node = ET.SubElement(parent, "node", attributes)
    return node


def add_node(parent, child_text, existing_nodes, link=None, color=None, style=None):
    """
    générer un noeud pour le titre parent

    """
    for child in parent.findall("node"):
        if child.get("TEXT") == child_text:
            return child  # retourner les noeuds existants
    return create_node(parent, child_text, link=link, color=color, style=style)  # retourner les nouveaux noeuds


def excel_to_freemind(dataframe):
    """
    convertir en freemind Tree
    """
    root = ET.Element("map", {"version": "1.0.1"}) # c'est la version de freeMind
    root_values = dataframe.iloc[:, 0].dropna().unique()  # récupérer les valeurs de la 1er colonne dans l'excel pour faire un noeud de mère

    # créer des noeuds pour chaque valeur dans la 1er colonne (normalement on a les valeurs identiques dans 1a 1er colonne)
    # si on a des valeurs différentes dans la 1er colonne, les noeuds seront créés pour les différent valeurs
    for root_value in root_values:
        main_node = create_node(root, str(root_value), color="#0000FF", style="bubble")  # styler le root noeud

        # 
        filtered_rows = dataframe[dataframe.iloc[:, 0] == root_value]

        # ajouter les noeuds de fils pour chaque ligne 
        for _, row in filtered_rows.iterrows():
            current_parent = main_node
            existing_nodes = set()
            for col_value in row[1:]:  # procéder à partir de la 2eme colonne
                if pd.isna(col_value):
                    continue  # skip les valeurs vides
                url_candidate = str(col_value)
                is_url = url_candidate.startswith("http://") or url_candidate.startswith("https://")
                if is_url:
                    # vérifier si url exists. si oui, set url as current noeud
                    current_parent.set("LINK", url_candidate)  
                else:
                    current_parent = add_node(
                        current_parent,
                        str(col_value),
                        existing_nodes,
                        color="#FF4500", # set la couleur des noeuds fils
                        style="bezier"   # set le style des liens entre noeuds mère et noeuds fils
                    )
    return root


def save_freemind_xml(root, output_file_path):
    """sauvegarder freeMind tree en fichier .mm"""
    tree = ET.ElementTree(root)
    tree.write(output_file_path, encoding="utf-8", xml_declaration=True)

In [9]:

# input data provenant de l'excel
# file_path = r"C:\Users\jl0331el\Documents\python\side_project\myplantonline/exported_data_2.xlsx"
df = pd.read_excel(input_file_path)

# générer le freeMind tree en xml (.mm)
freemind_tree = excel_to_freemind(df)

# sauvegarder le fichier .mm
# output_path = r"C:\Users\jl0331el\OneDrive - EDF\XMind/output_data2.mm"
save_freemind_xml(freemind_tree, output_file_path)